# 損切オーダーテスト

In [9]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_finance
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import schedule
import time
import threading
import json

In [2]:
# OANDA API v20の口座IDとAPIトークン(藤沼ver)
accountID = "101-009-12789646-001"
access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

In [3]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [4]:
def get_data(count, gran, year, month, day, hour, minute, second):
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [5]:
# 第何何曜日（第nX曜日）の日付を返す関数
def get_day_of_nth_dow(year, month, nth, dow):
    '''dow: Monday(0) - Sunday(6)'''
    if nth < 1 or dow < 0 or dow > 6:
        return None

    first_dow, n = calendar.monthrange(year, month)
    day = 7 * (nth - 1) + (dow - first_dow) % 7 + 1

    return day if day <= n else None

In [6]:
def get_data_super(start, end, gran):
    try:
        # OANDA API v20の口座IDとAPIトークン(藤沼ver)
        accountID = "101-009-12789646-001"
        access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
        # OANDAのデモ口座へのAPI接続
        api = API(access_token=access_token, environment="practice")
        # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
        s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
        e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
        fmt = '%Y-%m-%dT%H:%M:00.000000Z'
        from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                                  hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
        to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                                hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
        to_dt = '{}-{}-{} {}:{}:{}'.format(e_year, e_month, e_day, e_hour, e_minute, e_second)
        to_unix = datetime.datetime.strptime(to_dt, '%Y-%m-%d %H:%M:%S').timestamp()
        df = pd.DataFrame()
        # 5000個制限に引っかからなければこっちの処理
        try:
            params = {
                "granularity": gran,
                'from': from_,
                'to': to_
            }
            r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
            api.request(r)
            df = to_dataframe(r)
        # 引っかかればこっちの処理
        except:
            i = True
            while i:
                df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                                 hour=s_hour, minute=s_minute, second=s_second)
                # ケツが同じなら止める (最新迄用)
                if df.tail(1).reset_index(drop=True).equals(df_tmp.tail(1).reset_index(drop=True)):
                    i = False
                else:
                    df = pd.concat([df, df_tmp[:-1]], ignore_index=True)
                    index = df_tmp.tail(1).index[0]
                    s_year = df_tmp.iat[index, 0].year
                    s_month = df_tmp.iat[index, 0].month
                    s_day = df_tmp.iat[index, 0].day
                    s_hour = df_tmp.iat[index, 0].hour
                    s_minute = df_tmp.iat[index, 0].minute
                    s_second = df_tmp.iat[index, 0].second
                    # 5000個ずつ取得してエンドの unix 超えたら止める
                    if to_unix < df.iat[df.tail(1).index[0], 0].timestamp():
                        # はみ出した分をここで切り落とす
                        for i in range(len(df.index) - 5000, len(df.index)):
                            if to_unix <= df.iat[i, 0].timestamp():
                                df = df[:i]
                                break
                        i = False
    except ConnectionError:
        time.sleep(5)
        get_data_super(start, end, gran)
    return df

## 口座情報の取得

In [7]:
# OANDA API v20の口座IDとAPIトークン(藤沼ver)
accountID = "101-009-12789646-001"
access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")
s = accounts.AccountSummary(accountID)
api.request(s)

{'account': {'guaranteedStopLossOrderMode': 'DISABLED',
  'hedgingEnabled': False,
  'id': '101-009-12789646-001',
  'createdTime': '2019-11-26T09:10:00.745193269Z',
  'currency': 'JPY',
  'createdByUserID': 12789646,
  'alias': 'Primary',
  'marginRate': '0.04',
  'lastTransactionID': '31',
  'balance': '2999994.1358',
  'openTradeCount': 0,
  'openPositionCount': 0,
  'pendingOrderCount': 0,
  'pl': '-5.8642',
  'resettablePL': '-5.8642',
  'resettablePLTime': '0',
  'financing': '0.0000',
  'commission': '0.0000',
  'dividendAdjustment': '0',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '0.0000',
  'NAV': '2999994.1358',
  'marginUsed': '0.0000',
  'marginAvailable': '2999994.1358',
  'positionValue': '0.0000',
  'marginCloseoutUnrealizedPL': '0.0000',
  'marginCloseoutNAV': '2999994.1358',
  'marginCloseoutMarginUsed': '0.0000',
  'marginCloseoutPositionValue': '0.0000',
  'marginCloseoutPercent': '0.00000',
  'withdrawalLimit': '2999994.1358'},
 'lastTransactionID': '3

## 決済

In [36]:
position_data = { "longUnits": "ALL" }
p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
api.request(p)
p.response

{'longOrderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '-3',
  'timeInForce': 'FOK',
  'positionFill': 'REDUCE_ONLY',
  'reason': 'POSITION_CLOSEOUT',
  'longPositionCloseout': {'instrument': 'USD_JPY', 'units': 'ALL'},
  'id': '49',
  'accountID': '101-009-12609641-001',
  'userID': 12609641,
  'batchID': '49',
  'requestID': '24617877358095536',
  'time': '2019-11-22T03:07:24.252384609Z'},
 'longOrderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '49',
  'instrument': 'USD_JPY',
  'units': '-3',
  'requestedUnits': '-3',
  'price': '108.619',
  'pl': '0.0020',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '2998261.1548',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.0060',
  'fullVWAP': '108.619',
  'reason': 'MARKET_ORDER_POSITION_CLOSEOUT',
  'tradesClosed': [{'tradeID': '44',
    'units': '-1',
    'realizedPL': '-0.0

## 現在価格の取得

In [10]:
params = {"instruments": "USD_JPY"}
pi = PricingInfo(accountID=accountID, params=params)
api.request(pi)

{'time': '2020-01-12T12:50:52.533807970Z',
 'prices': [{'type': 'PRICE',
   'time': '2020-01-11T01:38:09.032101131Z',
   'bids': [{'price': '109.453', 'liquidity': 250000}],
   'asks': [{'price': '109.489', 'liquidity': 250000}],
   'closeoutBid': '109.445',
   'closeoutAsk': '109.497',
   'status': 'non-tradeable',
   'tradeable': False,
   'unitsAvailable': {'default': {'long': '250000', 'short': '250000'},
    'openOnly': {'long': '250000', 'short': '250000'},
    'reduceFirst': {'long': '250000', 'short': '250000'},
    'reduceOnly': {'long': '0', 'short': '0'}},
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'USD_JPY'}]}

In [14]:
# ロング価格
float(pi.response['prices'][0]['asks'][0]['price'])

109.489

In [15]:
# ショート価格
float(pi.response['prices'][0]['bids'][0]['price'])

109.453

## 現在のポジションを取得

In [43]:
# OANDA API v20の口座IDとAPIトークン(藤沼ver)
accountID = "101-009-12789646-001"
access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")
p = positions.OpenPositions(accountID=accountID)
api.request(p)

{'positions': [], 'lastTransactionID': '43'}

## オーダー

### 成り行き

In [98]:
data = {
  "order": {
    "instrument": "USD_JPY",
    "units": "+1",
    "type": "MARKET",
    "positionFill": "DEFAULT"
  }
}
o = orders.OrderCreate(accountID, data=data)
api.request(o)
o.response

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '1',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'reason': 'CLIENT_ORDER',
  'id': '55',
  'accountID': '101-009-12609641-001',
  'userID': 12609641,
  'batchID': '55',
  'requestID': '24617943972983195',
  'time': '2019-11-22T07:32:06.276523698Z'},
 'orderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '55',
  'instrument': 'USD_JPY',
  'units': '1',
  'requestedUnits': '1',
  'price': '108.578',
  'pl': '0.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '2998261.1808',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.0020',
  'fullVWAP': '108.578',
  'reason': 'MARKET_ORDER',
  'tradeOpened': {'price': '108.578',
   'tradeID': '56',
   'units': '1',
   'guaranteedExecutionFee': '0.0000',
   'halfSpreadCost': '0.0020',
   'initialMarginRequired': '4.3430'},
  

### 損切
シグナル点灯 => 現在価格の取得 => 損切ラインの算出 => オーダー

In [39]:
# 現在価格の取得
accountID = "101-009-12789646-001"
access_token = "8c5facf670b1061d19fe2c9a0874663f-3fcf75eb5ab3078ad10a63015e2d22d7"
api = API(access_token=access_token, environment="practice")

params = {"instruments": "USD_JPY"}
pi = PricingInfo(accountID=accountID, params=params)
api.request(pi)
# ロング価格
long_p = float(pi.response['prices'][0]['asks'][0]['price'])
# ショート価格
short_p = float(pi.response['prices'][0]['bids'][0]['price'])

In [40]:
# 損切ラインの算出(c pips)
fee = 0.002
c = 10
long_c = round(long_p - fee - c/100, 3)
short_c = round(short_p + fee + c/100, 3)
long_c, short_c

(109.522, 109.722)

In [41]:
# オーダー
stopLossOnFill = StopLossDetails(price=short_c)
ordr = MarketOrderRequest(
    instrument="USD_JPY",
    units=-10,
    stopLossOnFill=stopLossOnFill.data
)
o = orders.OrderCreate(accountID, data=ordr.data)
api.request(o)

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '-10',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'stopLossOnFill': {'price': '109.722', 'timeInForce': 'GTC'},
  'reason': 'CLIENT_ORDER',
  'id': '38',
  'accountID': '101-009-12789646-001',
  'userID': 12789646,
  'batchID': '38',
  'requestID': '42651119405470821',
  'time': '2020-01-13T03:04:53.472551547Z'},
 'orderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '38',
  'instrument': 'USD_JPY',
  'units': '-10',
  'requestedUnits': '-10',
  'price': '109.620',
  'pl': '0.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '2999994.0458',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.0200',
  'fullVWAP': '109.620',
  'reason': 'MARKET_ORDER',
  'tradeOpened': {'price': '109.620',
   'tradeID': '39',
   'units': '-10',
   'guaranteedExecutionFee': '0.0000',


## 過去ポジションの取得

In [48]:
import oandapyV20.endpoints.trades as trades
r = trades.TradeDetails(accountID, tradeID='39')
api.request(r)
r.response

{'trade': {'id': '39',
  'instrument': 'USD_JPY',
  'price': '109.620',
  'openTime': '2020-01-13T03:04:53.472551547Z',
  'initialUnits': '-10',
  'initialMarginRequired': '43.8488',
  'state': 'CLOSED',
  'currentUnits': '0',
  'realizedPL': '-0.0400',
  'closingTransactionIDs': ['42'],
  'financing': '0.0000',
  'dividendAdjustment': '0.0000',
  'closeTime': '2020-01-13T03:05:12.664421513Z',
  'averageClosePrice': '109.624',
  'stopLossOrder': {'id': '40',
   'createTime': '2020-01-13T03:04:53.472551547Z',
   'type': 'STOP_LOSS',
   'tradeID': '39',
   'price': '109.722',
   'guaranteed': False,
   'timeInForce': 'GTC',
   'triggerCondition': 'DEFAULT',
   'state': 'CANCELLED',
   'cancellingTransactionID': '43',
   'cancelledTime': '2020-01-13T03:05:12.664421513Z'}},
 'lastTransactionID': '43'}

In [51]:
r.response['trade']['realizedPL']

'-0.0400'